<a href="https://colab.research.google.com/github/Krishnaugale353/Pnemonia-Detection-using-ML/blob/main/Pnemonia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow-gpu


In [1]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob


In [2]:
IMAGESHAPE = [224, 224, 3]
training_data = '/content/drive/MyDrive/chest_xray/train'
testing_data = "/content/drive/MyDrive/chest_xray/test"


In [3]:
!nvidia-smi

Mon Mar  4 06:37:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)


In [5]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False


In [6]:
classes = glob('/content/drive/MyDrive/chest_xray/test/*')


In [7]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [8]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [10]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [11]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
												target_size = (224, 224),
												batch_size = 6,
												class_mode = 'categorical')


Found 5218 images belonging to 2 classes.


In [12]:
test_set = testing_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
											target_size = (224, 224),
											batch_size = 6,
											class_mode = 'categorical')


Found 624 images belonging to 2 classes.


In [13]:
print("Size of the training dataset:", len(training_set))
print("Size of the testing dataset:", len(test_set))

Size of the training dataset: 870
Size of the testing dataset: 104


In [ ]:
fitted_model = final_model.fit_generator(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)


<ipython-input-15-0133b02f9370>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator(


Epoch 1/5
870/870 [==============================] - 830s 949ms/step - loss: 0.2366 - accuracy: 0.9289 - val_loss: 0.4551 - val_accuracy: 0.8942
Epoch 2/5
870/870 [==============================] - 147s 169ms/step - loss: 0.2311 - accuracy: 0.9456 - val_loss: 1.3278 - val_accuracy: 0.8125
Epoch 3/5
870/870 [==============================] - 147s 169ms/step - loss: 0.1681 - accuracy: 0.9565 - val_loss: 1.2968 - val_accuracy: 0.8157
Epoch 4/5
870/870 [==============================] - 149s 172ms/step - loss: 0.1635 - accuracy: 0.9592 - val_loss: 0.7867 - val_accuracy: 0.8718
Epoch 5/5
870/870 [==============================] - 147s 169ms/step - loss: 0.1625 - accuracy: 0.9617 - val_loss: 0.4813 - val_accuracy: 0.9103


In [ ]:
final_model.save('our_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('our_model.h5') #Loading our model
img=image.load_img('/content/drive/MyDrive/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')


1/1 [==============================] - 2s 2s/step
Person is affected with Pneumonia.
Predictions: [[3.6420514e-34 1.0000000e+00]]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


# Evaluate the model on the testing set
evaluation = final_model.evaluate(test_set, steps=len(test_set))

# Print overall accuracy
print("Testing Accuracy: {:.2f}%".format(evaluation[1] * 100))

# Make predictions on the test set
predictions = final_model.predict(test_set, steps=len(test_set))

# Convert predictions to binary values (0 or 1)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = test_set.classes

# Print confusion matrix and classification report
conf_matrix = confusion_matrix(true_classes, predicted_classes)
classification_rep = classification_report(true_classes, predicted_classes, target_names=classes)

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_rep)


104/104 [==============================] - 10s 97ms/step - loss: 0.4813 - accuracy: 0.9103
Testing Accuracy: 91.03%
104/104 [==============================] - 9s 86ms/step

Confusion Matrix:
[[ 90 144]
 [154 236]]

Classification Report:
                                                  precision    recall  f1-score   support

   /content/drive/MyDrive/chest_xray/test/NORMAL       0.37      0.38      0.38       234
/content/drive/MyDrive/chest_xray/test/PNEUMONIA       0.62      0.61      0.61       390

                                        accuracy                           0.52       624
                                       macro avg       0.49      0.49      0.49       624
                                    weighted avg       0.53      0.52      0.52       624



In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Assuming you have the training_set and test_set from your CNN code

# Extract images and labels from the training set
train_images, train_labels = training_set.next()

# Reshape the data
train_images_flattened = train_images.reshape(train_images.shape[0], -1)

# Create and train the binary SVM model for normal vs. pneumonia
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=2))
svm_model.fit(train_images_flattened, np.argmax(train_labels, axis=1))

# Evaluate the model on the testing set
test_images, test_labels = test_set.next()
test_images_flattened = test_images.reshape(test_images.shape[0], -1)

predictions_svm = svm_model.predict(test_images_flattened)

# Print confusion matrix and classification report for SVM
conf_matrix_svm = confusion_matrix(np.argmax(test_labels, axis=1), predictions_svm)
classification_rep_svm = classification_report(np.argmax(test_labels, axis=1), predictions_svm)

print("\nSVM - Confusion Matrix:")
print(conf_matrix_svm)

print("\nSVM - Classification Report:")
print(classification_rep_svm)


SVM - Confusion Matrix:
[[4 1]
 [0 1]]

SVM - Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.50      1.00      0.67         1

    accuracy                           0.83         6
   macro avg       0.75      0.90      0.78         6
weighted avg       0.92      0.83      0.85         6



In [22]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Initialize empty lists to store images and labels
train_images_flattened = []
train_labels = []

# Loop through all batches in the training set and extract images and labels
for images_batch, labels_batch in training_set:
    # Flatten and append images
    train_images_flattened.extend(images_batch.reshape(images_batch.shape[0], -1))
    # Append labels
    train_labels.extend(labels_batch)

# Break the loop after iterating through all images
if len(train_images_flattened) == len(training_set.filenames):
    break

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images_flattened, train_labels, test_size=0.2, random_state=42)


In [ ]:
# Create and train the binary SVM model for normal vs. pneumonia
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=2))
svm_model.fit(X_train, y_train)

In [ ]:
# Evaluate on the validation set
predictions_val = svm_model.predict(X_val)
conf_matrix_val = confusion_matrix(y_val, predictions_val)
classification_rep_val = classification_report(y_val, predictions_val)

print("\nSVM Validation - Confusion Matrix:")
print(conf_matrix_val)

print("\nSVM Validation - Classification Report:")
print(classification_rep_val)

# Optional: Evaluate on the testing set (if provided)
if test_set:
    test_images, test_labels = test_set.images, test_set.labels
    test_images_flattened = test_images.reshape(test_images.shape[0], -1)
    predictions_svm = svm_model.predict(test_images_flattened)

    conf_matrix_svm = confusion_matrix(test_labels, predictions_svm)
    classification_rep_svm = classification_report(test_labels, predictions_svm)

    print("\nSVM - Confusion Matrix:")
    print(conf_matrix_svm)

    print("\nSVM - Classification Report:")
    print(classification_rep_svm)